In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# https://github.com/Protean-Labs/subgrounds
from subgrounds.subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField
from datetime import date, datetime, timedelta

sg = Subgrounds()

In [29]:
# SushiSwap theGraph node 1
sushi_1 = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/sushiswap/exchange')

# # SushiSwap theGraph node fork 2 from dev docs
# sushi_2 = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/zippoxer/sushiswap-subgraph-fork')

In [30]:
timestamp_90d = int(round((datetime.today() - timedelta(90)).timestamp()))

print(timestamp_90d)

1641244625


In [52]:
sushi_1.Pair.datetime = SyntheticField(
    f = lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    type_ = SyntheticField.STRING,
    deps = sushi_1.Pair.timestamp
)

timestamp_90d = int(round((datetime.today() - timedelta(90)).timestamp()))

## Current date provider count
lp_positions = sushi_1.Query.pairs(
    first = 1000000,
    orderBy = 'timestamp',
    orderDirection = 'desc',
    where = {
        'id': '0x6a091a3406e0073c3cd6340122143009adac0eda'
        # 'timestamp_gt': timestamp_90d
    }
)

lp_df = sg.query_df([ 
    lp_positions.datetime,
    lp_positions.id,
    lp_positions.name,
    lp_positions.liquidityProviderCount,
    lp_positions.liquidityPositions.id
])

In [53]:
lp_df.head()

,pairs_datetime,pairs_id,pairs_name,pairs_liquidityProviderCount,pairs_liquidityPositions_id
0,2021-04-02 18:25:40,0x6a091a3406e0073c3cd6340122143009adac0eda,ILV-WETH,0,0x6a091a3406e0073c3cd6340122143009adac0eda-0x
1,2021-04-02 18:25:40,0x6a091a3406e0073c3cd6340122143009adac0eda,ILV-WETH,0,0x6a091a3406e0073c3cd6340122143009adac0eda-0x0...
2,2021-04-02 18:25:40,0x6a091a3406e0073c3cd6340122143009adac0eda,ILV-WETH,0,0x6a091a3406e0073c3cd6340122143009adac0eda-0x0...
3,2021-04-02 18:25:40,0x6a091a3406e0073c3cd6340122143009adac0eda,ILV-WETH,0,0x6a091a3406e0073c3cd6340122143009adac0eda-0x0...
4,2021-04-02 18:25:40,0x6a091a3406e0073c3cd6340122143009adac0eda,ILV-WETH,0,0x6a091a3406e0073c3cd6340122143009adac0eda-0x0...


In [54]:
lp_df.pairs_datetime.max()

'2021-04-02 18:25:40'

In [37]:
## Synthetic field to transfer timestamp to datetime
sushi_1.user.datetime = SyntheticField(
    f = lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    type_ = SyntheticField.STRING,
    deps = sushi_1.user.timestamp
)

timestamp_90d = int(round((datetime.today() - timedelta(90)).timestamp()))

## Current date provider count
user_lp_positions = sushi_1.Query.users(
    first = 100000,
    # orderBy = 'timestamp',
    orderDirection = 'desc',
    where = {
        'liquidityTokenBalance_gt': 0.0000000001,
        # 'timestamp_gt': timestamp_90d,
        # 'pair_in': ['0xb84c45174bfc6b8f3eaecbae11dee63114f5c1b2'],
        # 'user_in': ['0xdb23c6e1ba73acf067956a41c57eeeb06374370e']
    }
)

In [38]:
user_lp_df = sg.query_df([ 
    user_lp_positions.datetime,
    user_lp_positions.user.id,
    user_lp_positions.pair.id,
    user_lp_positions.liquidityTokenBalance
])

user_lp_df.rename(columns={
    'liquidityPositions_datetime':'datetime',
    'liquidityPositions_user_id':'user_id',
    'liquidityPositions_pair_id':'pair_id',
    'liquidityPositions_liquidityTokenBalance':'lp_token_balance'
})

user_lp_df.head()

,liquidityPositions_datetime,liquidityPositions_user_id,liquidityPositions_pair_id,liquidityPositions_liquidityTokenBalance
0,2022-04-03 21:27:00,0xdb23c6e1ba73acf067956a41c57eeeb06374370e,0xb84c45174bfc6b8f3eaecbae11dee63114f5c1b2,105.648746


In [36]:
user_lp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2034 entries, 0 to 2033
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   liquidityPositions_datetime               2034 non-null   object 
 1   liquidityPositions_user_id                2034 non-null   object 
 2   liquidityPositions_pair_id                2034 non-null   object 
 3   liquidityPositions_liquidityTokenBalance  2034 non-null   float64
dtypes: float64(1), object(3)
memory usage: 63.7+ KB
